# bsh-tides-to-ical

Convert the [BSH tidal predictions](https://www.bsh.de/EN/DATA/Predictions/Tides/tides_node.html) for a calendar year from the downloaded txt file (HW/NW) into an iCalendar (.ics) file.



Example file
```
I_E#DE__715P2024#
A01#Hersteller :#M1103/BSH-Hamburg, 02.12.2022  09:57:37#
A02#Daten-Art  :#Vorausberechnungen#
A03#PegelNr.   :#DE__715P#
A04#GT-Name    :#Hamburg, Blankenese, Elbe, Unterfeuer#
A06#GT-Jahr    :#      2024#
A07#Def.-Jahr  :#2021#19#
A08#Position   :#53°33'28''N   9°47'45''E WGS84#
A11#Zeitzone   :#UTC+ 1h00min (MEZ)#
A12#Position GK:# 35 52809.00 R  59 36546.82 H#
A13#Messstelle :#307     #
C01#Analyse-Ber:#selbstständig#        #39 39#
C02#Datenumfang:#Zeiten u. Höhen: HW NW      #
C03#Höhenniveau:#PNP#
C04#EinheitZeit:#StdMin#
C05#EinheitHöhe:# m#
C12#VB-Methode :#HDdU (HW/NW)#
D01#PNP u. NHN :#- 5.02 #
D02#SKN u. NHN :#- 1.92 #
D03#SKN ü. PNP :#  3.10 #
F01#MHWI       :#15:03   #
F02#MNWI       :#22:01   #
G01#MHW        :#  7.02 #
G02#MNW        :#  3.43 #
LLL#
VB2#DE__715P# #N#Mo# 1. 1.2024# 1:50# 3.70 # #  1#+ 1:00#  26112#   2#2460310.534880#
VB2#DE__715P# #H#Mo# 1. 1.2024# 7:10# 7.22 # #  1#+ 1:00#  26112#   3#2460310.757291#
VB2#DE__715P# #N#Mo# 1. 1.2024#14:23# 3.68 # #  1#+ 1:00#  26112#   4#2460311.057736#
VB2#DE__715P# #H#Mo# 1. 1.2024#19:41# 6.87 # #  1#+ 1:00#  26113#   1#2460311.278446#
VB2#DE__715P# #N#Di# 2. 1.2024# 2:22# 3.76 # #  2#+ 1:00#  26113#   2#2460311.556967#
VB2#DE__715P# #H#Di# 2. 1.2024# 7:44# 7.18 # #  2#+ 1:00#  26113#   3#2460311.780623#
VB2#DE__715P# #N#Di# 2. 1.2024#14:57# 3.73 # #  2#+ 1:00#  26113#   4#2460312.081017#
VB2#DE__715P# #H#Di# 2. 1.2024#20:17# 6.84 # #  2#+ 1:00#  26114#   1#2460312.303360#
VB2#DE__715P# #N#Mi# 3. 1.2024# 2:59# 3.79 # #  3#+ 1:00#  26114#   2#2460312.582404#
VB2#DE__715P# #H#Mi# 3. 1.2024# 8:22# 7.10 # #  3#+ 1:00#  26114#   3#2460312.806863#
...
```

In [2]:
from datetime import datetime, timedelta
from icalendar import Calendar, Event


def read_tide_data(file_path):
    '''read and extract high tide date and time'''
    high_tides = []
    gt_name = ''

    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('A04#'):
                gt_name = line.split('#')[2]
                
            if line.startswith('VB2#'):  # Zeilen mit Hochwasserdaten beginnen mit 'VB2#'
                data = line.split('#')
                if data[3] == 'H':  # Überprüfen, ob es sich um Hochwasser handelt
                    date_str = data[5] + ' ' + data[6]  # Datum und Uhrzeit extrahieren
                    try:
                        date_time = datetime.strptime(date_str, '%d. %m.%Y %H:%M')  # Datum und Uhrzeit parsen
                    except ValueError:
                        date_time = datetime.strptime(date_str, '%d.%m.%Y %H:%M')  # Versuch eines alternativen Formats
                    high_tides.append(date_time)  # Hochwasserzeit zur Liste hinzufügen

    return gt_name, high_tides

# Funktion zum Erstellen der ICS-Datei


def create_ics_file(high_tides):
    cal = Calendar()

    for tide in high_tides[1]:
        event = Event()
        event.add('summary', 'HW {} Uhr'.format(tide.strftime('%H:%M')))
        event.add('dtstart', tide - timedelta(hours = 2))
        event.add('dtend', tide + timedelta(hours = 2))
        event.add('location', high_tides[0])
        cal.add_component(event)

    output_file = high_tides[0].lower().replace(', ','_') + '_2024.ics'

    with open(output_file, 'wb') as f:
        f.write(cal.to_ical())
        
        
file_path = 'DE__715P2024.txt'
high_tides = read_tide_data(file_path)

# ICS-Datei erstellen
create_ics_file(high_tides)